# Pandas and Enhancements to Regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def readDat(fname):
    '''
    :param fname: input file
    Assumes a first comment line is attribute names
    Other rows are attribute values
    :returns: numpy array of data, one example per row.
    '''
    with open(fname,'r') as fd:
        dat = []
        attribs = None
        for line in fd.readlines():
            if line[0] == '#': #ignore comments
                attribs = line.split()[1:]
            else:
                x = line.split()
                vals = [ float(val) for val in x ]
                dat.append( vals ) # convert to floats and append
    return attribs,np.array(dat)

In [ ]:
attr,dat = readDat('auto.dat')

In [ ]:
dat

## Use Pandas to looks at your data!

In [ ]:
## Look at data
auto_dataframe = pd.DataFrame(dat,columns=attr)
pd.plotting.scatter_matrix(auto_dataframe,figsize=(15,15),marker='0',hist_kwds={'bins':20},s=60,alpha=0.8);

## Let's fit MPG vs. WEIGHT using linear regression.

In [ ]:
plt.scatter(dat[:,3],dat[:,-1])
plt.title('MPG vs. WEIGHT')

In [ ]:
def gradStep(dat,p):
    '''
    Calculate one update of the parameters in p via gradient descent
    algorithm.
    dat: all input data
    p: parameters of linear equation to be adjusted
    '''
    sum = np.zeros(2)
    # Variables are x = x[0:2], with x[0] always equal to 1.
    for x in dat:
        sum += (np.dot(pmat,x[0:2]) - x[2]) * x[0:2]
    return sum / len(dat)


def cost(dat,p):
    '''Cost function (sum of squares of differences)'''
    sum = 0
    for x in dat:
        sum += ( np.dot(x[0:2], p) - x[2] )**2
    return sum / (2.0 * len(dat))

In [ ]:
def gradStepStoch(dat,p,alpha):
    '''
    Calculate one update of the parameters in p via stochastic gradient descent
    algorithm.
    dat: all input data
    p: parameters of linear equation to be adjusted
    '''
    # For all data items sum term, (h(x) - y)*x 
    # Predicted value is y = x[2]
    # Variables are x = x[0:2], with x[0] always equal to 1.
    for x in dat:
        # It's critical to separate calculation of diff
        # from changes to parameters p!
        diff = np.dot(p,x[0:2]) - x[2]
        p[0] += -alpha * diff * x[0]
        p[1] += -alpha * diff * x[1]


In [ ]:
def regress(dat,limit=0.001,alpha=1e-8):
    '''Use linear regression to predict col2 from col1 in dat'''
    p = np.random.rand(2) - 0.5   # two random values ((these are the parameters to determine)
   
    newp = p.copy()
    
    err = cost(dat,p)
    print(err)
    olderr = 2 * err

    while olderr - err > limit: # terminate when error change is small
        gradStepStoch(dat,p,alpha) # calculate gradient step
        olderr = err
        err = cost(dat,p) # determine cost (error)
    print('Error',err,olderr)
    print ('Parameters: ', p)

In [ ]:
# Set up input data for two variable regression by prepending BIAS
nrows = len(dat[:,[3,-1]])
bias = np.ones((nrows,1))
regressDat = np.concatenate( (bias,dat[:,[3,-1]]),axis=1)

In [ ]:
regress(regressDat,1e-3,1e-9) # How do the estimated parameters look?

## Compare with sklearn

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
nrows = len(regressDat)
x = regressDat[:,1].reshape((nrows,1))
y = regressDat[:,2].reshape((nrows,1))
lr = LinearRegression().fit(x,y)
m = lr.coef_
b = lr.intercept_
print(lr.intercept_,lr.coef_) # intercept and slope

## Plot the data and the obtained line.

In [ ]:
x= regressDat[:,1]
plt.scatter(x,regressDat[:,2])
plt.plot(x, 46.2 + -0.0076*x)

## We clearly have a problem, right?

Let's clean up our data so that all variables have the same scale.  When variables are of different scales, the derivative of one can completely dominate the contribution of the other variables.

As it stands, some variables like weight have large values, whereas others, like the number of cylinders, have small values.  We can put all variables on equal footing if we convert them to z scores $z = \frac{(x-\mu)}{\sigma}$

In [ ]:
from sklearn import preprocessing
from numpy.linalg import inv

class rescale:
    '''Rescale vars and provide inverse.
        Assumes zeroth column is all ones, the bias.
    '''
    def __init__(self,dat):
        self.dat = dat
        self.means = dat.mean(axis=0)
        self.stdevs = dat.std(axis=0)
        if 0 in self.stdevs[1:]:
            raise Exception("Zero stdev does not permit transform.")
        
    def scale(self):
        '''Return the data columns scaled to each have 
        zero mean and stdev of 1.  The zeroth column is
        not scaled.
        '''
        self.scaled = preprocessing.scale(self.dat)
        self.scaled[:,0] = self.dat[:,0] # Copy back the bias column
        return self.scaled
    
    def transform(self,x):
        '''Transform x[1:] to new zscore.  Does not change x[0].'''
        z = np.zeros(len(x))
        z[0] = x[0]
        for i in range(1,len(x)):
            z[i] = (x[i] - self.means[i]) / self.stdevs[i]
        return z
                    
    def untransform(self,z):
        '''Transform z to score in original space.  Does not change z[0].'''
        x = np.zeros(len(z))
        x[0] = z[0]
        for i in range(1,len(z)):
            x[i] = self.means[i] + (self.stdevs[i] * z[i])
        return x
    
  

In [ ]:
scaler = rescale(regressDat)
scaledDat = scaler.scale()
print('means',np.mean(scaledDat,axis=0))
print('deviations',np.std(scaledDat,axis=0))

In [ ]:
#Check with SciKitlearn
nrows = len(regressDat)
x = scaledDat[:,1].reshape((nrows,1))
y = scaledDat[:,2].reshape((nrows,1))
lr = LinearRegression().fit(x,y)
m = lr.coef_
b = lr.intercept_
print('b and m',b,m)

In [ ]:
regress(scaledDat,1e-10,1e-5)

### Note
The values we derive are in the renormalized space.  When testing data, we must preserve the mean and standard deviation of the __training data__ so we can renormalize the testing data in the same manner.

## Regularization

When we fit numerous parameters, we would like to avoid overfitting and we may believe that some parameters are more important predictors than others.  Looking at the scatterplots above, which predictors have the most obvious relationships to the MPG we want to predict?

One approach to this problem is to enforce a constraint on all parameters that we add to the loss function , $J(\theta)$.  In particular we use
$
J(\theta) = \sum_{i=1}^{m} ((\sum_{k=1}^{n} \theta_k x^{(i)}_k) - y^{(i)}) + \lambda \sum_{k=1}^{n} \theta_k^2
$
Of course, we choose a weighting value, $\lambda$ and we must still decide whether we want to use stochastic gradient descent.

## Rewrite our code a bit and include regularization.

In [ ]:
def cost(dat,y,p,regwt=0.0):
    '''Cost function (sum of squares of differences).
        Implements L2 regularization
    '''
    sum = 0
    for i,x in enumerate(dat):
        sum += ( np.dot(x, p) - y[i] )**2 + regwt * np.dot(p,p)
    return sum / (2.0 * len(dat))

def gradStepStoch(dat,y,p,alpha,regwt=0.0):
    '''
    Calculate one update of the parameters in p via stochastic gradient descent
    algorithm.  
    dat: all input data
    p: parameters of linear equation to be adjusted
    regwt: regularization parameter for L2 regularization
    '''
    # For all data items sum term, (h(x) - y)*x 
    # Predicted value is y = x[-1]
    # Variables are x = x[0:-1], with x[0] always equal to 1.
    for i,x in enumerate(dat):
        diff = np.dot(p,x) - y[i] 
        p += -alpha * diff * x
        

def regress(dat,y,regwt=0.0,alpha=1e-3,limit=1e-3):
    '''Use linear regression to predict col2 from col1 in dat'''
    nparams = np.shape(dat)[1] 
    p = np.random.rand(nparams) - 0.5 # two random values ((these are the parameters to determine)
 
    err = cost(dat,y,p)
    print(err)
    olderr = 2 * err

    while olderr - err > limit: # terminate when error change is small
        gradStepStoch(dat,y,p,alpha,regwt) # calculate gradient step
        olderr = err
        err = cost(dat,y,p) # determine cost (error)
    print('Error',err,olderr)

    print ('Parameters: ', p)


In [ ]:
# Check our old problem.
x = scaledDat[:,:-1]
y = scaledDat[:,-1]
regress(x,y,regwt=0.0,alpha=1e-3,limit=1e-6)

In [ ]:
# Now we rescale ALL data
nrows = len(dat)
bias = np.ones((nrows,1))
allDat = np.concatenate( (bias,dat),axis=1)
scaler = rescale(allDat)
allScaled = scaler.scale()

In [ ]:
np.shape(allDat)

In [ ]:
x = allScaled[:,:-1]
y = allScaled[:,-1]
regress(x,y,regwt=0.0,alpha=1e-4,limit=1e-10)

In [ ]:
rwt=0.9
regress(x,y,rwt,alpha=1e-5,limit=1e-9)
            

In [ ]:
#Check with SciKitlearn
nrows = len(x)
xx = allScaled[:,1:-1]
yy = allScaled[:,-1]
lr = LinearRegression().fit(xx,yy)
m = lr.coef_
b = lr.intercept_
print('b and m',b,m)

In [ ]:
allDat

In [ ]:
X = allDat[:,:-1]
I = np.eye( len(X.T) )
regwt = 0.0
Y = allDat[:,-1]

In [ ]:
np.shape(Y)

In [ ]:
M = np.matmul(X.T,X)
Minv = np.linalg.inv(M)
Q = np.dot(X.T,Y)
np.dot(Minv,Q)

In [ ]:
np.shape(M)

In [ ]:
np.shape(X)

In [ ]:
lr = LinearRegression().fit(X,Y)
m = lr.coef_
b = lr.intercept_
print('b and m',b,m)